7. 기상레이더 자료 분석을 위한 시각화

7-1. 기상레이더 자료 읽기

In [1]:
# 필요한 모듈 가져오기  
import numpy as np
import sys
import gzip # bin.gz 형식 파일 읽는데 사용
from netCDF4 import Dataset # netCDF 형식 파일 읽는데 사용


7-1-1. 레이더 강수량 자료 읽기

In [2]:
# 강수량 자료 읽는 함수(파일 형식: bin.gz)
def read_hsp_bin(file, varname):

    # 파일 존재 여부 확인
    try:
        status = open(file, 'r') # 파일 열기
    except:
        print('[Error] Open: ', file) # 에러 발생 시 출력
        sys.exit(2) # 프로그램 종료(2는 비정상 종료를 의미함)
    else:
        print('[Log] Open: ', file) # 에러가 없는 경우 출력
        status.close() # 파일 닫기

    # 바이트를 디코딩(decoding)하여 아스크코드로 변환하는 함수
    # ord(표 7-5), Str.decode(표 7-6) 설명 참고
    def bin2str(binary):
        return [ord(c) for c in binary.decode('ascii')]

    # 시간 문자열을 datetime.datetime() 형식으로 반환하는 함수
    # file.read(표 7-7), np.frombuffer(표 7-8) 설명 참고
    def timestr2dt(file):
        # 시간에 대한 구조체 읽기(TIME_SS: 7 bytes)
        yy = np.frombuffer(file.read(2), dtype=np.int16)[0]
        mm = ord(file.read(1))
        dd = ord(file.read(1))
        hh = ord(file.read(1))
        mi = ord(file.read(1))
        ss = ord(file.read(1))
        try:
            # datetime.datetime 형식으로 변경 
            return datetime.datetime(yy,mm,dd,hh,mi,ss)
        except:
            return -1
    
    # gzip 모듈 open()함수를 이용하여 자료 읽음
    # gzip.open(표 7-9) 설명 참고
    with gzip.open(file,'rb') as f:
        # 자료 헤더 읽기(RDR_CMP_HEA: 64 bytes)
        version = ord(f.read(1)) 
        ptype = np.frombuffer(f.read(2), dtype=np.int16)[0] 
        tm = timestr2dt(f) 
        tm_in = timestr2dt(f) 
        num_stn  = ord(f.read(1)) 
        map_code = ord(f.read(1)) 
        map_etc  = ord(f.read(1)) 
        nx  = np.frombuffer(f.read(2), dtype=np.int16)[0] 
        ny  = np.frombuffer(f.read(2), dtype=np.int16)[0] 
        nz  = np.frombuffer(f.read(2), dtype=np.int16)[0] 
        dxy = np.frombuffer(f.read(2), dtype=np.int16)[0] 
        dz  = np.frombuffer(f.read(2), dtype=np.int16)[0] 
        z_min = np.frombuffer(f.read(2), dtype=np.int16)[0] 
        num_data = ord(f.read(1)) 
        data_code = bin2str(f.read(16)) 
        etc = bin2str(f.read(15)) 
        
        # 자료 헤더 읽기(RDR_CMP_STN LIST: 20 bytes*48)
        for ii in range(48):
            # RDR_CMP_STN LIST(20 bytes)
            stn_cd = bin2str(f.read(6)) 
            _tm    = timestr2dt(f) 
            _tm_in = timestr2dt(f) 
        
        # 강수량, 고도정보, 지점정보를 1차원 배열로 읽음
        echo_1D = np.frombuffer(f.read(2*nx*ny), 
                    dtype=np.int16)
        topo_1D = np.frombuffer(f.read(2*nx*ny), 
                    dtype=np.int16)
        site_1D = np.frombuffer(f.read(2*nx*ny), 
                    dtype=np.int16)

    # 관측영역 밖 자료 nan 값 할당 및 scale factor 곱하기 
    if (varname == 'rain'):
        echo_1D = np.where(echo_1D==-30000, np.nan, echo_1D)
        _data = echo_1D * 0.01
    elif (varname == 'topo'):        
        topo_1D = np.where(topo_1D==-30000, np.nan, topo_1D)
        _data = topo_1D * 0.001
    elif (varname == 'stn'):
        site_1D = np.where(site_1D==-30000, np.nan, site_1D)
        _data = site_1D 

    # 1차원 배열을 2차원 배열(ny, nx)로 변환
    _data = _data.copy().reshape(ny, nx) 

    # 딕셔너리(dictionary) 생성
    data = {} 

    data['nx'] = nx
    data['ny'] = ny
    data['dxy'] = dxy/1000. # [km]
    data[varname] = _data

    return data


In [3]:
# 파일 읽기 예시
fpath = './DATA/' # 디렉토리 설정 필요
file = fpath + 'RDR_CMP_HSP_EXT_202208081730.bin.gz'
varname = 'rain'
hsp = read_hsp_bin(file, varname)

print(hsp.keys())
print(hsp['nx'])
print(hsp['ny'])
print(hsp['dxy'])



[Log] Open:  ./DATA/RDR_CMP_HSP_EXT_202208081730.bin.gz
dict_keys(['nx', 'ny', 'dxy', 'rain'])
2305
2881
0.5


7-1-2. 레이더 바람장 자료 읽기

In [4]:
# 바람장(WISSDOM) 자료 읽는 함수(파일 형식: netCDF)
def read_wissdom_nc(file, varname, is_level, level):
    
    # 파일 존재 여부 확인
    try:
        status = open(file, 'r') # 파일 열기
    except:
        print ('[Error] Open: ', file) # 에러 발생 시 출력
        sys.exit(2) # 프로그램 종료(2는 비정상 종료를 의미함)
    else:
        print ('[Log] Open: ', file) # 에러가 없는 경우 출력
        status.close() # 파일 닫기
    
    f = Dataset(file) # 파일 열기
    
    # 차원(dimensions) 정보 읽기
    # f.dimensions(표 7-10) 설명 참고
    nx = f.dimensions['nx'].size #960
    ny = f.dimensions['ny'].size #960
    nz = f.dimensions['nz'].size #56
    
    # 전역 속성(global attributes) 정보 읽기
    # f.getncattr(표 7-11) 설명 참고
    data_out = int(f.getncattr('data_out')) 
    data_minus = float(f.getncattr('data_minus'))
    data_scale = float(f.getncattr('data_scale'))
    grid_size = float(f.getncattr('grid_size')) #dxy
    sx = int(f.getncattr('map_sx')) 
    sy = int(f.getncattr('map_sy'))
    cen_lon = float(f.getncattr('map_slon'))
    cen_lat = float(f.getncattr('map_slat'))
    
    # 입력 변수명에 대한 변수(variables) 읽기
    # f.variables(표 7-12) 설명 참고
    height = f.variables['height'][:]
    if varname == 'u' :
        _data = f.variables['u_component'][:,:,:]
    elif varname == 'v' :
        _data = f.variables['v_component'][:,:,:]    
    elif varname == 'w' :
        _data = f.variables['w_component'][:,:,:]   
    elif varname == 'div' :
        _data = f.variables['divergence'][:,:,:]
    elif varname == 'ref' :
        _data = f.variables['reflectivity'][:,:,:]
    elif varname == 'vort' :    
        _data = f.variables['vertical_vorticity'][:,:,:]
    elif varname == 'vt' :        
        _data = f.variables['vertical_velocity'][:,:,:]
    else :
        print("there is no fieldname !!!")

    f.close() # 파일 닫기

    # 관측 영역 밖 자료 nan 값 할당
    _data = np.where(_data==data_out, np.nan, _data)

    # offset 뺀 후 scale factor 곱하기
    if varname != "div" or varname != "vort" :
        _data = (_data - data_minus) / data_scale
    else :
        _data  = np.where(_data<=0, 
            pow(10, (_data-data_minus)/data_scale),
            -1*pow(10, -1*(_data-data_minus)/data_scale)) 
    
    data = {} # 딕셔너리 생성
    
    if is_level == True : # 특정 고도 자료 추출
        z_index = np.where(height == level) # 고도 index 추출
        print("index at the given height: ", z_index)
        data['nx'] = nx
        data['ny'] = ny
        data['dxy'] = grid_size/1000. # [km]
        data[varname] = _data[z_index[0][0],:,:] # 2차원 배열
    elif is_level == False : # 모든 고도 자료 추출 
        data['nx'] = nx
        data['ny'] = ny
        data['nz'] = nz   
        data['dxy'] = grid_size/1000. # [km]
        data['height'] = height
        data[varname] = _data[:,:,:] # 3차원 배열

    return data



In [5]:
# 파일 읽기 예시
fpath = './DATA/' # 디렉토리 설정 필요
file = fpath + 'RDR_R3D_KMA_WD_202208081730_new.nc'

u = read_wissdom_nc(file, 'u', True, 3000)

print(u.keys())
print(u['nx'])
print(u['ny'])
print(u['dxy'])



[Log] Open:  ./DATA/RDR_R3D_KMA_WD_202208081730_new.nc
index at the given height:  (array([35]),)
dict_keys(['nx', 'ny', 'dxy', 'u'])
960
960
1.0


7-1-3. 레이더 반사도 합성장 자료 읽기

In [6]:
# 3차원 합성장(R3D) 자료 읽는 함수(파일 형식: netCDF)
def read_r3d_nc(file, varname, is_level, level):
    
    # 파일 존재 여부 확인
    try:
        status = open(file, 'r') # 파일 열기
    except:
        print ('[Error] Open: ', file) # 에러 발생 시 출력
        sys.exit(2) # 프로그램 종료(2는 비정상 종료를 의미함)
    else:
        print ('[Log] Open: ', file) # 에러가 없는 경우 출력
        status.close() # 파일 닫기
        
    f = Dataset(file) # 파일 열기 

    # 차원(dimensions) 정보 읽기
    nx = f.dimensions['nx'].size  #2049
    ny = f.dimensions['ny'].size  #2049
    nz = f.dimensions['nz'].size  #105
    
    # 전역 속성(global attributes) 정보 읽기
    grid_size = float(f.getncattr('grid_size')) #dxy
    sx = int(f.getncattr('map_sx'))
    sy = int(f.getncattr('map_sy'))
    cen_lon = float(f.getncattr('map_slon'))
    cen_lat = float(f.getncattr('map_slat'))
    
    # 입력 변수명에 대한 변수(variables) 읽기
    height = f.variables['height'][:]
    _data = f.variables['data'][:,:,:]
    var = f.variables['data']
    data_out = int(var.getncattr('data_out'))
    data_minus = float(var.getncattr('data_minus'))
    data_scale = float(var.getncattr('data_scale'))

    f.close() # 파일 닫기

    # 관측 영역 밖 자료 nan 값 할당
    _data = np.where(_data==data_out, np.nan, _data)

    # offset 뺀 후 scale factor 곱하기
    _data = (_data - data_minus) / data_scale

    data = {} # 딕셔너리 생성

    if is_level == True :
        z_index = np.where(height == level) # 고도 index 추출
        print("index at the given height: ", z_index)
        
        data['nx'] = nx
        data['ny'] = ny
        data['dxy'] = grid_size/1000. # [km]
        data[varname] = _data[z_index[0][0],:,:] # 2차원 배열
    elif is_level == False :
        data['nx'] = nx
        data['ny'] = ny
        data['nz'] = nz
        data['dxy'] = grid_size/1000. # [km]
        data['height'] = height
        data[varname] = _data[:,:,:] # 3차원 배열

    return data


In [7]:
# 파일 읽기 예시
fpath = './DATA/' # 디렉토리 설정 필요
file = fpath + 'RDR_R3D_EXT_CZ_202207110120_new.nc'

cz = read_r3d_nc(file, 'CZ', True, 1450)
print(cz.keys())
print(cz['nx'])
print(cz['ny'])
print(cz['dxy'])

[Log] Open:  ./DATA/RDR_R3D_EXT_CZ_202207110120_new.nc
index at the given height:  (array([14]),)
dict_keys(['nx', 'ny', 'dxy', 'CZ'])
2049
2049
0.5


7-2. 좌표변환 하기

In [8]:
# 필요한 모듈 가져오기  
import math
from math import asin, sin, cos, tan, log

In [9]:
# 좌표변환(위경도 to x, y 또는 x, y to 위경도)을 위한 클래스 생성
# 지도투영볍: Lambert conformal conic projection
class Lcc_proj():
    
    # 좌표변화에 필요한 변수(표 7-13) 초기화(initialization)
    def __init__(self, map_code: str, grid_size: float):
        self.Re = 6371.00877  # 지구반경[km]
        self.grid = grid_size # 격자크기[km]
        self.slat1 = 30.0 # 표준위도 1[degree]
        self.slat2 = 60.0 # 표준위도 2[degree]
        self.olon = 126.0 # 기준점 경도[degree]
        self.olat = 38.0  # 기준점 위도[degree]

        if map_code == 'HB' : # 강수량 자료
            print("map_code: HB, gridsize: "+str(grid_size))
            self.xo = 560 / self.grid # HB 영역 기준점 x좌표
            self.yo = 840 / self.grid # HB 영역 기준점 y좌표
        elif map_code == 'HW' : # 바람장 자료
            print("map_code: HW, gridsize: "+str(grid_size))
            self.xo = 440 / self.grid # HW 영역 기준점 x좌표
            self.yo = 700 / self.grid # HW 영역 기준점 y좌표
        elif map_code == 'HR' : # 반사도 자료
            print("map_code: HR, gridsize: "+str(grid_size))
            self.xo = 440 / self.grid # HR 영역 기준점 x좌표
            self.yo = 770 / self.grid # HR 영역 기준점 x좌표
        else:
            print("There is no map_code !!!") # 지도 코드 없음
   
        self.PI = asin(1.0) * 2.0 
        # 디그리(degree)를 라디안(radian)으로 변환
        self.DEGRAD = self.PI / 180.0
# 라디안(radian)을 디그리(degree)로 변환
        self.RADDEG = 180.0 / self.PI 

        self.re = self.Re / self.grid # 격자크기 기준으로 재설정
        self.slat1 = self.slat1 * self.DEGRAD
        self.slat2 = self.slat2 * self.DEGRAD
        self.olon = self.olon * self.DEGRAD
        self.olat = self.olat * self.DEGRAD

        self.sn = tan(self.PI * 0.25 + self.slat2 * 0.5)/ \
                    tan(self.PI * 0.25 + self.slat1 * 0.5)
        self.sn = log(cos(self.slat1) / cos(self.slat2))/ \
                    log(self.sn)
        self.sf = tan(self.PI * 0.25 + self.slat1 * 0.5)
        self.sf = pow(self.sf, self.sn) * cos(self.slat1)/ \
                    self.sn
        self.ro = tan(self.PI * 0.25 + self.olat * 0.5)
        self.ro = self.re*self.sf / pow(self.ro, self.sn)

    # 위경도를 (x, y) 격자점으로 바꾸는 함수
    def latlonToGrid(self, lat, lon):
        ra = math.tan(self.PI*0.25 + lat * self.DEGRAD*0.5)
        ra = self.re * self.sf / pow(ra, self.sn)
        theta = lon * self.DEGRAD - self.olon
        if theta > self.PI :
            theta -= 2.0 * self.PI
        if theta < -self.PI :
            theta += 2.0 * self.PI
        theta *= self.sn
        x = (ra * math.sin(theta)) + self.xo
        y = (self.ro - ra * math.cos(theta)) + self.yo
        
        x = int(x + 0.5)
        y = int(y + 0.5)

        return x, y

    # (x, y) 격자점을 위경도로 바꾸는 함수
    def gridToLatlon(self, x, y):
        xn = x - self.xo
        yn = self.ro - y + self.yo
        ra = math.sqrt(xn * xn + yn * yn)
        if self.sn < 0.0 :
            ra = -ra
        alat = math.pow((self.re*self.sf/ra), (1.0/self.sn))
        alat = 2.0 * math.atan(alat) - self.PI * 0.5
        if math.fabs(xn) <= 0.0 :
            theta = 0.0
        else :
            if math.fabs(yn) <= 0.0 :
                theta = self.PI * 0.5
                if xn < 0.0 :
                    theta = -theta
            else :
                theta = math.atan2(xn, yn)
        alon = theta / self.sn + self.olon
        lat = alat * self.RADDEG
        lon = alon * self.RADDEG

        return lat, lon


In [10]:
# 강수량 자료에 대한 좌표변환 예시
map = Lcc_proj("HB", 0.5)
x, y = map.latlonToGrid(38.0, 126.0)
print(x, y)
lat, lon = map.gridToLatlon(1120, 1680)
print(lat, lon)

map = Lcc_proj("HB", 1.0)
x, y = map.latlonToGrid(38.0, 126.0)
print(x, y)

lat, lon = map.gridToLatlon(560, 840)
print(lat, lon)


map_code: HB, gridsize: 0.5
1120 1680
38.0 126.0
map_code: HB, gridsize: 1.0
560 840
38.0 126.0


In [11]:
# 바람장 자료에 대한 좌표변환 예시
map = Lcc_proj("HW", 1.0)
x, y = map.latlonToGrid(38.0, 126.0)
print(x, y)

lat, lon = map.gridToLatlon(440, 700)
print(lat, lon)


map_code: HW, gridsize: 1.0
440 700
38.0 126.0


In [12]:
# 반사도 자료에 대한 좌표변환 예시
map = Lcc_proj("HR", 0.5)
x, y = map.latlonToGrid(38.0, 126.0)
print(x, y)

lat, lon = map.gridToLatlon(880, 1540)
print(lat, lon)


map_code: HR, gridsize: 0.5
880 1540
38.0 126.0


7-3. 사용자 정의 색상표 만들기

In [13]:
# 필요한 모듈 가져오기  
import numpy as np
from matplotlib import colors

In [14]:
# 사용자 정의 색상표(colorbar)를 만드는 함수 
def custom_colormap(varname):

    # RGB 값이 저장된 파일 위치
    rgb_path = './RES/'    

    # RGB 값이 저장된 파일명
    if varname=="CZ" :
        colrfile = rgb_path + 'color_rdr_DZ.rgb'
    elif varname=="rain":
        colrfile = rgb_path + 'color_rdr_echo.rgb'
    elif varname=="u" or varname=="U" : 
        colrfile = rgb_path + 'color_rdr_U.rgb'     
    else :
        print('there is no field type !!!')        
    
    # RGB 값 파일명 출력
    print('RGB file: ', colrfile) 

    # RGB 값이 저장된 파일 읽기
    red   = np.genfromtxt(colrfile, usecols=0, dtype='int')
    green = np.genfromtxt(colrfile, usecols=1, dtype='int')
    blue  = np.genfromtxt(colrfile, usecols=2, dtype='int')
    
    # RGB 값이 0~1 사이 값을 가지도록 정규화(normalization)
    cdict = {'red':[],'green':[],'blue':[]} # 딕셔너리 초기화
    position = np.linspace(0,1,len(red)) 
    for pos, color in zip(position, red) : 
        cdict['red'].append((pos, color/256., color/256.))
    for pos, color in zip(position, green) :
        cdict['green'].append((pos, color/256., color/256.))
    for pos, color in zip(position, blue) :
        cdict['blue'].append((pos, color/256., color/256.))
    
    # 선형 보간을 통한 색상표 객체(cmap) 생성,
    # 눈금(level), 긴 변수명(long_name), 단위(unit) 설정
    # colors. LinearSegmentedColormap(표 7-14) 설명 참고
    if varname=="CZ":
        cmap=colors.LinearSegmentedColormap(
            'DZ', cdict, len(red))
        level = [-80,-20,-10,-5,0,5,10,12,14,16,18,20,22,\
            24,26,28,30,32,34,36,38,40,42,44,46,48,50,55,\
            60,65,70,80,100]
        long_name = 'Reflectivity'
        unit = '[dBZ]'
    elif varname == "rain":
        cmap=colors.LinearSegmentedColormap(
            'RN', cdict, len(red))
        level = [-300,0.01,0.1,0.5,1.0,2.0,3.0,4.0,5.0,\
            6.0,7.0,8.0,9.0,10.0,15.0,20.0,25.0,30.0,  \
            40.0,50.0,60.0,70.0,90.0,110.0,150.0,200.0]    
        long_name = 'Rainrate'
        unit = r'[mm $h^{-1}$]'     
    elif varname=='u' or varname =='U' :
        cmap=colors.LinearSegmentedColormap(
            'U', cdict, len(red)) 
        level = [-60,-30,-15,-10,-8,-6,-4,-2,0.0, \
            2,4,6,8,10,15,30,60]     
        long_name = 'Wind speed(U)'
        unit = '[m/s]'    
    else : 
        print('there is no field type !!!')   

    # 색상표 딕셔너리 초기화 및 생성 
    # colors.BoundaryNorm(표 7-15) 설명 참고
    dict_cmap = {}
    dict_cmap['cmap'] = cmap
    dict_cmap['norm'] = colors.BoundaryNorm(level, \
                            ncolors=len(level))
    dict_cmap['ticks'] = level[1:-1]
    dict_cmap['vmin']  = level[1]
    dict_cmap['vmax']  = level[-1]
    dict_cmap['long_name'] = long_name
    dict_cmap['unit'] = unit

    return dict_cmap


In [15]:
# 강수량 색상표 
cmap = custom_colormap("rain")

print(cmap.keys())
print(cmap['ticks'])
print(cmap['vmin'])
print(cmap['vmax'])
print(cmap['long_name'])
print(cmap['unit'])


RGB file:  ./RES/color_rdr_echo.rgb
dict_keys(['cmap', 'norm', 'ticks', 'vmin', 'vmax', 'long_name', 'unit'])
[0.01, 0.1, 0.5, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 15.0, 20.0, 25.0, 30.0, 40.0, 50.0, 60.0, 70.0, 90.0, 110.0, 150.0]
0.01
200.0
Rainrate
[mm $h^{-1}$]


7-4. 레이더 자료 표출하기 

In [16]:
# 필요한 모듈 가져오기  
import os
# 지도상 표출에 필요한 모듈
from matplotlib import pyplot as plt
from mpl_toolkits.basemap import Basemap
# 연직 단면도 표출에 필요한 모듈
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker

7-4-1. 레이더 자료 지도상 표출하기 

In [23]:
# Basemap을 이용한 지도상 표출하기 
def draw_rdr_product_basemap(tm, nx, ny, data, product_name, 
    field_name, level, is_wind, is_AB, A_latlon, B_latlon):

    # 그림 객체 생성
    fig = plt.figure(figsize=(10,10))
    ax = plt.axes()

    # 좌표변환을 위한 지도 영역 설정
    if product_name=='HSP' :  
        map = Lcc_proj("HB", 0.5)
    elif product_name=='WISSDOM' :
        map = Lcc_proj("HW", 1.0)
    elif product_name=="R3D" :
        map = Lcc_proj("HR", 0.5)
    else :
        print("no product type !!!")

    # 격자점에서의 위경도 계산
    lon = np.zeros((ny, nx), float) # lon
    lat = np.zeros((ny, nx), float) # lat
    for j in range(0, ny):
        for i in range(0, nx):
            lat[j,i], lon[j,i] = map.gridToLatlon(i, j)

    # Basemap을 이용한 지도 설정(지도 투영법: LCC)
    m = Basemap(llcrnrlon=lon[0,0], llcrnrlat=lat[0,0], 
        urcrnrlon=lon[ny-1,nx-1], urcrnrlat=lat[ny-1,nx-1],
        lat_1=30., lat_2=60., lon_0=126.0, lat_0=38.0, 
        projection='lcc', resolution='h') 

    # 해안선, 국경, 위경도선 그리기
    m.drawcoastlines(linewidth=0.5) # 해안선
    m.drawcountries(linewidth=0.5)  # 국경
    m.drawmeridians(np.arange(120., 140., 2.), \
        labels=[False, False, False, True]) # 경도선 
    m.drawparallels(np.arange(30., 60., 2.), \
        labels=[True, False, False, False]) # 위도선

    # shape 파일을 이용한 남한 영역 행정구역 그리기
    # readshapefile(표 7-16) 설명 참고 
    shape_fn = './RES/' + \
        'shapefile/TL_SCCO_CTPRVN_WGS84_UTF8' # shape 파일명
    m.readshapefile(shape_fn, 'CTPRVN', linewidth=0.4)

    cmap=custom_colormap(varname=field_name) # 색상표

    # pcolormesh 이용한 자료 표출, colorbar 이용한 색상표 표출
    x, y = m(lon, lat)
    cs = m.pcolormesh(x, y, data, 
        cmap=cmap['cmap'], norm=cmap['norm']) # 자료 표출
    cb = m.colorbar(cs,
        cmap=cmap['cmap'], ticks=cmap['ticks']) # 색상표 표출
    cb.set_label(cmap['unit']) # 색상표 단위 표출

    # 그림 타이틀 생성
    if is_wind == True :
        tit = cmap["long_name"]+cmap["unit"]+' '+ \
            tm+'(KST)'+' \n'+ \
            'wind(vector)'+'@'+str(level/1000.)+'km'
    else :
        if product_name=='WISSDOM' or product_name=='R3D' :
            tit = cmap["long_name"]+cmap["unit"]+' '+ \
                '@'+str(level/1000.)+'km'+tm+'(KST)'
        else :
            tit = cmap["long_name"]+cmap["unit"]+' '+ \
                tm+'(KST)'
    plt.title(tit, fontsize=14, loc='left') # 그림 타이틀 삽입

    # 연직 단면의 시작점과 끝점(A-B) 지점 그리기
    if is_AB == True :
        AB_lon = [A_latlon[1], B_latlon[1]]
        AB_lat = [A_latlon[0], B_latlon[0]]
        xAB_lon, xAB_lat = m(AB_lon, AB_lat)
        
        # 두 지점을 연결하는 선 그리기
        m.plot(AB_lon, AB_lat, 'r^', markersize=15)
        m.plot(xAB_lon, xAB_lat, color='r', linewidth=3)

        # 두 지점에 ‘A’와 ‘B’ 텍스트 삽입하기
        ax.annotate('A', (xAB_lon[0], xAB_lat[0]), \
            xytext=(5, 5), textcoords='offset points', \
            color='r', fontsize=16, weight='bold')
        ax.annotate('B', (xAB_lon[1], xAB_lat[1]), \
            xytext=(5, 5), textcoords='offset points', \
            color='r', fontsize=16, weight='bold')

    # 바람 벡터 중첩하기
    fn_wind= ''
    if is_wind == True :
        # 특정고도에서의 바람장 자료 추출
        fpath = './DATA/' # 디렉토리 설정 필요
        file = fpath + 'RDR_R3D_KMA_WD_'+tm+'_new.nc'
        u = read_wissdom_nc(file, 'u', True, level) # u 성분
        v = read_wissdom_nc(file, 'v', True, level) # v 성분

        map = Lcc_proj("HW", 1.0) # 도메인 영역 설정

        # gridToLatlon을 이용한 위경도 계산하기
        nx = u['nx']
        ny = u['ny']

        lon = np.zeros((ny, nx), float) # lon
        lat = np.zeros((ny, nx), float) # lat
        for j in range(0, ny):
            for i in range(0, nx):
                lat[j,i], lon[j,i] = map.gridToLatlon(i, j)

        # quiver를 이용한 바람 벡터 그리기
        v_int = 40
        xx, yy = m(lon, lat)      
        cv = m.quiver(xx[::v_int,::v_int], \
            yy[::v_int,::v_int], \
            list(u['u'][::v_int,::v_int]), \
            list(v['v'][::v_int,::v_int]),\
            pivot='mid',scale_units='inches', \
            scale=70, units='xy')

        # quiverkey를 이용한 바람장 기준 벡터 추가하기 
        cvec = ax.quiverkey(cv, 0.9, 1.025, 10, \
            r'10 $ms^{-1}$', labelpos='E', labelsep=0.05)
        
        # 파일 이름에 바람 추가
        fn_wind= '_wUVvector'

    # 이미지 파일 이름 설정 및 저장 
    img_path = './FIGS/'
    img_fn = img_path+'RDR_%s_%s_%s_basemap%s.png' \
        %(product_name, field_name, tm, fn_wind)
    print('IMG file: '+img_fn)
    plt.savefig(img_fn, format='png')
    plt.close()


In [24]:
# 강수량(HSP) 자료 표출 예시 
tm = '202208081730'
fpath = './DATA/' # 디렉토리 설정 필요
file = fpath + 'RDR_CMP_HSP_EXT_'+tm+'.bin.gz'
varname = 'rain'
level = 950

hsp = read_hsp_bin(file, varname)

# 강수량 그림 그리기 w/o 바람벡터
draw_rdr_product_basemap(tm, hsp['nx'], hsp['ny'], \
    hsp[varname], "HSP", varname, level, \
    False, False, [-999.0, -999.0], [-999.0, -999.0])


[Log] Open:  ./DATA/RDR_CMP_HSP_EXT_202208081730.bin.gz
map_code: HB, gridsize: 0.5
[[120.16721732 120.17240678 120.17759629 ... 132.15443177 132.15961805
  132.16480429]
 [120.1668386  120.1720284  120.17721824 ... 132.15483121 132.16001783
  132.1652044 ]
 [120.16645983 120.17164996 120.17684014 ... 132.15523071 132.16041766
  132.16560457]
 ...
 [118.82754297 118.83391279 118.84028271 ... 133.56720516 133.57356909
  133.57993293]
 [118.82697081 118.83334114 118.83971155 ... 133.56780844 133.57417288
  133.58053722]
 [118.82639856 118.83276939 118.83914031 ... 133.56841182 133.57477676
  133.5811416 ]]
[[30.14675521 30.14708254 30.14740958 ... 30.12589382 30.1255482
  30.1252023 ]
 [30.15124278 30.15157014 30.1518972  ... 30.13037963 30.13003398
  30.12968805]
 [30.15573043 30.15605782 30.15638491 ... 30.13486552 30.13451985
  30.13417389]
 ...
 [43.32304046 43.32345648 43.32387214 ... 43.29652734 43.29608811
  43.29564851]
 [43.32767446 43.32809052 43.32850621 ... 43.30115914 43.300

/var/folders/dm/6bh9yvs11nvf3wn5xmhtwksr0000gq/T/ipykernel_58105/2501029318.py:52: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  cs = m.pcolormesh(x, y, data,


IMG file: ./FIGS/RDR_HSP_rain_202208081730_basemap.png


In [19]:
# 강수량 그림 그리기 w/ 바람벡터
print(level)
draw_rdr_product_basemap(tm, hsp['nx'], hsp['ny'], \
    hsp[varname], "HSP", varname, level, \
    True, False, [-999.0, -999.0], [-999.0, -999.0])


950
map_code: HB, gridsize: 0.5
RGB file:  ./RES/color_rdr_echo.rgb


/var/folders/dm/6bh9yvs11nvf3wn5xmhtwksr0000gq/T/ipykernel_58105/893716685.py:50: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  cs = m.pcolormesh(x, y, data,


[Log] Open:  ./DATA/RDR_R3D_KMA_WD_202208081730_new.nc
index at the given height:  (array([19]),)
[Log] Open:  ./DATA/RDR_R3D_KMA_WD_202208081730_new.nc
index at the given height:  (array([19]),)
map_code: HW, gridsize: 1.0
IMG file: ./FIGS/RDR_HSP_rain_202208081730_basemap_wUVvector.png


In [20]:
# 바람장(WISSDOM) 자료 표출 예시 
tm = '202207110120'
fpath = './DATA/' # 디렉토리 설정 필요
file = fpath + 'RDR_R3D_KMA_WD_'+tm+'_new.nc'
varname = 'u'
level = 1400

wissdom = read_wissdom_nc(file, varname, True, level)

# 바람장 자료 표출 w/ 바람벡터 + A-B 지점 표출하기
A_latlon = [34.0000, 125.5000]
B_latlon = [32.5000, 124.5000]

draw_rdr_product_basemap(tm, wissdom['nx'], wissdom['ny'], \
    wissdom[varname], "WISSDOM", varname, level, \
    True, True, A_latlon, B_latlon)


[Log] Open:  ./DATA/RDR_R3D_KMA_WD_202207110120_new.nc
index at the given height:  (array([24]),)
map_code: HW, gridsize: 1.0
RGB file:  ./RES/color_rdr_U.rgb
[Log] Open:  ./DATA/RDR_R3D_KMA_WD_202207110120_new.nc


/var/folders/dm/6bh9yvs11nvf3wn5xmhtwksr0000gq/T/ipykernel_58105/893716685.py:50: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  cs = m.pcolormesh(x, y, data,


index at the given height:  (array([24]),)
[Log] Open:  ./DATA/RDR_R3D_KMA_WD_202207110120_new.nc
index at the given height:  (array([24]),)
map_code: HW, gridsize: 1.0
IMG file: ./FIGS/RDR_WISSDOM_u_202207110120_basemap_wUVvector.png


7-4-2. 레이더 자료 연직 단면도 표출하기 

In [21]:
# 연직 단면 표출 함수 
def draw_rdr_product_cross_section(tm, dxy, data, height,
    product_name, field_name, is_wind, A_latlon, B_latlon):
    
    # 좌표변환을 위한 지도 영역 설정
    if product_name == 'HSP' :  
        map = Lcc_proj("HB", 0.5)
    elif product_name == 'WISSDOM' :
        map = Lcc_proj("HW", 1.0)
    elif product_name == "R3D" :
        map = Lcc_proj("HR", 0.5)
    else :
        print("no product type !!!")

    # A, B 지점 x축 격자점, y축 격자점 추출
    x1, y1 = map.latlonToGrid(A_latlon[0], A_latlon[1])
    x2, y2 = map.latlonToGrid(B_latlon[0], B_latlon[1])
    print('Start point=>', x1, y1, ', End point=>', x2, y2)

    # 두 점(A-B) 사이의 격자점(xl, yl) 계산 및 x축 값(xf) 추출
    # length(x1, x2 혹은 y1, y2 사이 구간의 정밀도) 정의
    length = max(np.abs(x1-x2), np.abs(y1-y2)) 
    xl = np.linspace(x1, x2, length).astype(int) # x축 격자점
    yl = np.linspace(y1, y2, length).astype(int) # y축 격자점
    xx = np.sqrt((x1-x2)**2+(y1-y2)**2)*dxy #거리
    if x1 < x2 :
        xf = np.linspace(0, np.floor(xx), length) # x축 값
    else :
        xf = np.linspace(np.floor(xx), 0, length) # x축 값

    # 두 점(A-B) 사이의 자료 추출
    m_data = data[:,yl,xl]

    # 그림 객체 생성
    fig = plt.figure(figsize=(10, 10))
    ax = plt.axes()
    plt.ylim(0.0, 10.0)

    # x축, y축에 대한 라벨 설정
    if x1 < x2 :
        plt.xlabel("Distance from A to B [km]", fontsize=16) 
    else :
        plt.xlabel("Distance from B to A [km]", fontsize=16)
    plt.ylabel("Height [km]", fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    cmap=custom_colormap(varname=field_name) # 색상표 

    # pcolormesh 이용한 자료 표출 및 colorbar 이용한 색상표 표출 
    height = np.round(height*1e-3, 3)
    ap = ax.pcolormesh(xf, height, m_data,
        cmap=cmap['cmap'], norm=cmap['norm'])
    cax = ax.inset_axes([1.01, 0., 0.03, 1.]) # 색상표 위치설정
    cb = fig.colorbar(ap, cax=cax, cmap=cmap['cmap'], \
        norm=cmap['norm'], ticks=cmap['ticks']) # 색상표 표출
    cb.set_label(cmap['unit'], fontsize=12) # 색상표 단위 표출

    # x축과 y축에 대한 눈금 간격을 설정
    ax.xaxis.set_major_locator(ticker.MultipleLocator(xx/10)) 
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1.0)) 
    
    # 그림 타이틀 삽입
    latlon_A_to_B = "A(%6.2f" % (A_latlon[1])+r'$^{o}$N'+ \
        ", "+ "%6.2f" % (A_latlon[0])+r'$^{o}$E' ") to "+ \
        "B(%6.2f" % (B_latlon[1])+r'$^{o}$N'+ \
        ", %6.2f" % (B_latlon[0])+r'$^{o}$E'+")"
    tit = cmap["long_name"]+cmap["unit"]+' '+tm+'(KST)'+ \
        ' \n'+latlon_A_to_B
    plt.title(tit, fontsize=14, loc='left')       

    # 바람 벡터 중첩 
    if is_wind == True :
        # 3차원 바람장 자료 추출
        fpath = './DATA/' # 디렉토리 설정 필요
        file = fpath + 'RDR_R3D_KMA_WD_'+tm+'_new.nc'
        u = read_wissdom_nc(file, 'u', False, 0) # u 성분
        v = read_wissdom_nc(file, 'v', False, 0) # v 성분
        
        height = np.round(u['height']*1e-3, 3) # 고도 정보

        map = Lcc_proj("HW", 1.0) # 지도 영역 설정

        # A, B 지점 x축 격자점, y축 격자점 추출
        x1, y1 = map.latlonToGrid(A_latlon[0], A_latlon[1])
        x2, y2 = map.latlonToGrid(B_latlon[0], B_latlon[1])
        print('Start point=>', x1,y1, ', End point=>', x2,y2)

        # 두 점(A-B) 사이의 격자점 계산 및 x축 값 추출
        length = 12 # x1, x2 혹은 y1, y2 사이 구간의 정밀도
        xl = np.linspace(x1,x2,length).astype(int) #x축 격자점
        yl = np.linspace(y1,y2,length).astype(int) #y축 격자점
        xx = np.sqrt((x1-x2)**2+(y1-y2)**2)*u['dxy'] # 거리
        if x1 < x2 :
            xf = np.linspace(0, np.floor(xx), length) #x축 값
        else :
            xf = np.linspace(np.floor(xx), 0, length) #x축 값

        # 두 점(A-B) 지점 사이의 자료 추출
        uu = u['u'][:,yl,xl]
        vv = v['v'][:,yl,xl]
      
        xf_2d, height_2d = np.meshgrid(xf, height) 

        # 특정 고도 간격으로 자료 추출
        # 바람장의 고도 간격은 가변이므로 고도별로 간격을 다르게 추출
        slt_xf_2d = np.concatenate( 
            (xf_2d[0:21:4, ::], xf_2d[21:31:2, ::], \
            xf_2d[31:46:1, ::],  xf_2d[46:56:1, ::] ) ) 
        slt_height_2d = np.concatenate( 
            (height_2d[0:21:4, ::], height_2d[21:31:2, ::],\
            height_2d[31:46:1, ::], height_2d[46:56:1, ::]) )
        slt_uu = np.concatenate( 
            ([uu[0:21:4, ::], uu[21:31:2, ::], \
            uu[31:46:1, ::], uu[46:56:1, ::]]) )
        slt_vv = np.concatenate( 
            ([vv[0:21:4, ::], vv[21:31:2, ::], \
            vv[31:46:1, ::], vv[46:56:1, ::]]) )

        # quiver를 이용한 바람 벡터 그리기
        cv = plt.quiver(slt_xf_2d[::,1:-1:1], 
            slt_height_2d[::,1:-1:1], 
            slt_uu[::,1:-1:1], slt_vv[::,1:-1:1],
            pivot='mid', scale_units='inches', 
            scale=80, units='xy')   

        # quiverkey를 이용한 바람장 기준 벡터 추가하기
        cvec = ax.quiverkey(cv, 0.9, 1.025, 10, 
            r'10 $ms^{-1}$', labelpos='E', labelsep=0.05)

    # 이미지 파일 이름 설정 및 저장 
    img_path = './FIGS/'
    img_fn = img_path+'RDR_%s_%s_%s_cross_section.png' \
        %(product_name, field_name, tm)
    print('IMG file: '+img_fn)
    plt.savefig(img_fn, format='png')
    plt.close()


In [22]:
# 3차원 반사도 자료 표출 예시  
tm = '202207110120'
fpath = './DATA/' # 디렉토리 설정 필요
file = fpath + 'RDR_R3D_EXT_CZ_'+tm+'_new.nc'
varname = 'CZ'
level = 1450

cz = read_r3d_nc(file, varname, False, level)

A_latlon = [34.0000, 125.5000]
B_latlon = [32.5000, 124.5000]

# 반사도 자료 표출 w/ 바람벡터
draw_rdr_product_cross_section(tm, cz['dxy'], \
    cz[varname], cz['height'], 'R3D', \
    varname, True, A_latlon, B_latlon)


[Log] Open:  ./DATA/RDR_R3D_EXT_CZ_202207110120_new.nc
map_code: HR, gridsize: 0.5
Start point=> 789 669 , End point=> 601 342
RGB file:  ./RES/color_rdr_DZ.rgb
[Log] Open:  ./DATA/RDR_R3D_KMA_WD_202207110120_new.nc
[Log] Open:  ./DATA/RDR_R3D_KMA_WD_202207110120_new.nc
map_code: HW, gridsize: 1.0
Start point=> 395 265 , End point=> 301 101
IMG file: ./FIGS/RDR_R3D_CZ_202207110120_cross_section.png
